# Key frame finding 
The algorithm has been adapted from [pedrofrodenas/blur-Detection-Haar-Wavelet](https://github.com/pedrofrodenas/blur-Detection-Haar-Wavelet)

In [1]:
import sys
import pywt
import cv2
import numpy as np
import os
import argparse
import json

In [2]:
def blur_detect(img, threshold):
    
    # Convert image to grayscale
    Y = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    M, N = Y.shape
    
    # Crop input image to be 3 divisible by 2
    Y = Y[0:int(M/16)*16, 0:int(N/16)*16]
    
    # Step 1, compute Haar wavelet of input image
    LL1,(LH1,HL1,HH1)= pywt.dwt2(Y, 'haar')
    # Another application of 2D haar to LL1
    LL2,(LH2,HL2,HH2)= pywt.dwt2(LL1, 'haar') 
    # Another application of 2D haar to LL2
    LL3,(LH3,HL3,HH3)= pywt.dwt2(LL2, 'haar')
    
    # Construct the edge map in each scale Step 2
    E1 = np.sqrt(np.power(LH1, 2)+np.power(HL1, 2)+np.power(HH1, 2))
    E2 = np.sqrt(np.power(LH2, 2)+np.power(HL2, 2)+np.power(HH2, 2))
    E3 = np.sqrt(np.power(LH3, 2)+np.power(HL3, 2)+np.power(HH3, 2))
    
    M1, N1 = E1.shape

    # Sliding window size level 1
    sizeM1 = 8
    sizeN1 = 8
    
    # Sliding windows size level 2
    sizeM2 = int(sizeM1/2)
    sizeN2 = int(sizeN1/2)
    
    # Sliding windows size level 3
    sizeM3 = int(sizeM2/2)
    sizeN3 = int(sizeN2/2)
    
    # Number of edge maps, related to sliding windows size
    N_iter = int((M1/sizeM1)*(N1/sizeN1))
    
    Emax1 = np.zeros((N_iter))
    Emax2 = np.zeros((N_iter))
    Emax3 = np.zeros((N_iter))
    
    
    count = 0
    
    # Sliding windows index of level 1
    x1 = 0
    y1 = 0
    # Sliding windows index of level 2
    x2 = 0
    y2 = 0
    # Sliding windows index of level 3
    x3 = 0
    y3 = 0
    
    # Sliding windows limit on horizontal dimension
    Y_limit = N1-sizeN1
    
    while count < N_iter:
        # Get the maximum value of slicing windows over edge maps 
        # in each level
        Emax1[count] = np.max(E1[x1:x1+sizeM1,y1:y1+sizeN1])
        Emax2[count] = np.max(E2[x2:x2+sizeM2,y2:y2+sizeN2])
        Emax3[count] = np.max(E3[x3:x3+sizeM3,y3:y3+sizeN3])
        
        # if sliding windows ends horizontal direction
        # move along vertical direction and resets horizontal
        # direction
        if y1 == Y_limit:
            x1 = x1 + sizeM1
            y1 = 0
            
            x2 = x2 + sizeM2
            y2 = 0
            
            x3 = x3 + sizeM3
            y3 = 0
            
            count += 1
        
        # windows moves along horizontal dimension
        else:
                
            y1 = y1 + sizeN1
            y2 = y2 + sizeN2
            y3 = y3 + sizeN3
            count += 1
    
    # Step 3
    EdgePoint1 = Emax1 > threshold;
    EdgePoint2 = Emax2 > threshold;
    EdgePoint3 = Emax3 > threshold;
    
    # Rule 1 Edge Pojnts
    EdgePoint = EdgePoint1 + EdgePoint2 + EdgePoint3
    
    n_edges = EdgePoint.shape[0]
    
    # Rule 2 Dirak-Structure or Astep-Structure
    DAstructure = (Emax1[EdgePoint] > Emax2[EdgePoint]) * (Emax2[EdgePoint] > Emax3[EdgePoint]);
    
    # Rule 3 Roof-Structure or Gstep-Structure
    
    RGstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax1[i] < Emax2[i] and Emax2[i] < Emax3[i]:
            
                RGstructure[i] = 1
                
    # Rule 4 Roof-Structure
    
    RSstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax2[i] > Emax1[i] and Emax2[i] > Emax3[i]:
            
                RSstructure[i] = 1

    # Rule 5 Edge more likely to be in a blurred image 

    BlurC = np.zeros((n_edges));

    for i in range(n_edges):
    
        if RGstructure[i] == 1 or RSstructure[i] == 1:
        
            if Emax1[i] < threshold:
            
                BlurC[i] = 1                        
        
    # Step 6
    Per = np.sum(DAstructure)/np.sum(EdgePoint)
    
    # Step 7
    if (np.sum(RGstructure) + np.sum(RSstructure)) == 0:
        
        BlurExtent = 100
    else:
        BlurExtent = np.sum(BlurC) / (np.sum(RGstructure) + np.sum(RSstructure))
    
    return Per, BlurExtent

def find_images(input_dir):
    extensions = [".jpg", ".png", ".jpeg"]

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in extensions:
                yield os.path.join(root, file)
                

In [3]:
# parser = argparse.ArgumentParser(description='run Haar Wavelet blur detection on a folder')
# parser.add_argument('-i', '--input_dir', dest="input_dir", type=str, required=True, help="directory of images")
# parser.add_argument('-s', '--save_path', dest='save_path', type=str, help="path to save output")
# parser.add_argument("-t", "--threshold", dest='threshold', type=float, default=35, help="blurry threshold")
# parser.add_argument("-d", "--decision", dest='MinZero', type=float, default=0.001, help="MinZero Decision Threshold")
# args = parser.parse_args()
''' -- ORIGINAL VERSION --
input_dir = "videos/keyframes"  # Change this to your image directory
save_path = "videos/result.json"  # Change this to your desired output JSON file
threshold = 35
minZero = 0.001
    
results = []

for input_path in find_images(input_dir):
    try:
        I = cv2.imread(input_path)
        per, blurext = blur_detect(I, threshold)
        if per < minZero:
            classification = True
        else:
            classification = False
        results.append({"input_path": input_path, "per": per, "blur extent": blurext, "is blur": classification})
        print("{0}, Per: {1:.5f}, blur extent: {2:.3f}, is blur: {3}".format(input_path, per, blurext, classification))
        
        
    except Exception as e:
        print(e)
        pass
    
if save_path:
    
    assert os.path.splitext(save_path)[1] == ".json", "You must include the extension .json on the end of the save path"
    
    with open(save_path, 'w') as outfile:
        json.dump(results, outfile, sort_keys=True, indent=4)
        outfile.write("\n")
'''

' -- ORIGINAL VERSION --\ninput_dir = "videos/keyframes"  # Change this to your image directory\nsave_path = "videos/result.json"  # Change this to your desired output JSON file\nthreshold = 35\nminZero = 0.001\n    \nresults = []\n\nfor input_path in find_images(input_dir):\n    try:\n        I = cv2.imread(input_path)\n        per, blurext = blur_detect(I, threshold)\n        if per < minZero:\n            classification = True\n        else:\n            classification = False\n        results.append({"input_path": input_path, "per": per, "blur extent": blurext, "is blur": classification})\n        print("{0}, Per: {1:.5f}, blur extent: {2:.3f}, is blur: {3}".format(input_path, per, blurext, classification))\n        \n        \n    except Exception as e:\n        print(e)\n        pass\n    \nif save_path:\n    \n    assert os.path.splitext(save_path)[1] == ".json", "You must include the extension .json on the end of the save path"\n    \n    with open(save_path, \'w\') as outfile:

In [5]:
input_dir = "videos/keyframes"  # Change this to your image directory
save_path = "videos/result.json"  # Change this to your desired output JSON file
threshold = 35
minZero = 0.001
    
results = []
best_result = 1.0
best_result_path = ""
for input_path in find_images(input_dir):
    try:
        I = cv2.imread(input_path)
        per, blurext = blur_detect(I, threshold)
        # if per < minZero:
        #     classification = True
        # else:
        #     classification = False
        if blurext < best_result:
            best_result = blurext
            best_result_path = input_path
        
    except Exception as e:
        print(e)
        pass
    


print (best_result)
print (best_result_path)

0.8303008466942893
videos/keyframes/24.jpg
